In [85]:
import os
import pandas as pd
import requests
import json

from elasticsearch import Elasticsearch

In [86]:
# Get players

players = pd.read_json("game-changers/esports-data/players.json", dtype={'id': str, 'home_team_id': str})
teams = pd.read_json("game-changers/esports-data/teams.json", dtype={'id': str})
teams.rename(columns={'id': 'team_id'}, inplace=True)

players = pd.merge(players, teams[['team_id', 'name', 'slug', 'acronym']], left_on='home_team_id', right_on='team_id', how='left')

# Rename columns to clean up
players.rename(columns={'name': 'team_name', 'slug': 'team_slug', 'acronym': 'team_acronym'}, inplace=True)

players

,id,handle,first_name,last_name,status,photo_url,home_team_id,created_at,updated_at,team_id,team_name,team_slug,team_acronym
0,107025876564296044,Toma,Tommi,Lehtinen,active,http://static.lolesports.com/players/163303303...,106652238530500027,2021-10-01 11:19:00+00:00,2021-10-01 11:19:00+00:00,106652238530500027,Rix.gg Lightning,rixgg-lightning,RIX
1,107025876564296044,Toma,Tommi,Lehtinen,active,http://static.lolesports.com/players/163303303...,106652238530500027,2021-10-01 11:19:00+00:00,2021-10-01 11:19:00+00:00,106652238530500027,Rix.gg Lightning,rixgg-lightning,RIX
2,106977714184829995,Stefanie,Stefanie,Jones,active,http://static.lolesports.com/players/163235037...,106976771652907805,2021-09-22 22:41:26+00:00,2023-04-09 14:50:55+00:00,106976771652907805,Dignitas,dignitas,DIG
3,107176790303905255,aRth,Arthur,Hardman,active,None,107174817570297246,2021-10-28 14:16:57+00:00,2021-11-01 13:52:31+00:00,107174817570297246,Breakout Esports,breakout-esports,OUT
4,107282855606202828,godwana,Engin,Balcı,active,http://static.lolesports.com/players/163700646...,107021298845350518,2021-11-15 21:00:23+00:00,2023-05-29 08:05:32+00:00,107021298845350518,Futbolist JEFF,futbolist-female,FUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5947,112793696105404261,WINT3R,winter,winter,active,https://s3.us-west-2.amazonaws.com/static.lole...,112251458972768983,2024-07-16 02:09:10+00:00,2024-07-21 14:04:21+00:00,112251458972768983,Team Leveling,team-leveling,TLV
5948,112793696105404261,WINT3R,winter,winter,active,https://s3.us-west-2.amazonaws.com/static.lole...,112251458972768983,2024-07-16 02:09:10+00:00,2024-07-21 14:04:21+00:00,112251458972768983,Team Leveling,team-leveling,TLV
5949,112251720337473925,natasha05,Giulia,Natasha,active,https://s3.us-west-2.amazonaws.com/static.lole...,109993966808522027,2024-07-20 00:39:24+00:00,2024-07-20 00:39:24+00:00,109993966808522027,LOUD,loud,LOUD
5950,112251720337473925,natasha05,Giulia,Natasha,active,https://s3.us-west-2.amazonaws.com/static.lole...,109993966808522027,2024-07-20 00:39:24+00:00,2024-07-20 00:39:24+00:00,109993966808522027,LOUD,loud,LOUD


In [87]:
players.dtypes

id                           object
handle                       object
first_name                   object
last_name                    object
status                       object
photo_url                    object
home_team_id                 object
created_at      datetime64[ns, UTC]
updated_at      datetime64[ns, UTC]
team_id                      object
team_name                    object
team_slug                    object
team_acronym                 object
dtype: object

In [88]:
# Check if an ID exist in dataset
players[players['id'] == "108764522500018946"]

,id,handle,first_name,last_name,status,photo_url,home_team_id,created_at,updated_at,team_id,team_name,team_slug,team_acronym
2454,108764522500018946,SayaJP,Sayaka,Kojima,active,http://static.lolesports.com/players/164373420...,108764409368519858,2022-08-04 12:12:30+00:00,2024-07-08 03:07:07+00:00,108764409368519858,REIGNITE Lily,reignite-lily,RIG
2455,108764522500018946,SayaJP,Sayaka,Kojima,active,http://static.lolesports.com/players/164373420...,108764409368519858,2022-08-04 12:12:30+00:00,2024-07-08 03:07:07+00:00,108764409368519858,REIGNITE Lily,reignite-lily,RIG
2456,108764522500018946,SayaJP,Sayaka,Kojima,active,http://static.lolesports.com/players/164373420...,108764409368519858,2022-08-04 12:12:30+00:00,2024-07-08 03:07:07+00:00,108764409368519858,REIGNITE Lily,reignite-lily,RIG


In [89]:
# Select player
target_name = "SayaJP"

player_id = str(list(set(players[players['handle'] == target_name]['id']))[0])

player_id

'108764522500018946'

In [90]:
# Get all player entries that a player played for (Dataset IDs are not the same?)
players[players['handle'] == target_name]

,id,handle,first_name,last_name,status,photo_url,home_team_id,created_at,updated_at,team_id,team_name,team_slug,team_acronym
2454,108764522500018946,SayaJP,Sayaka,Kojima,active,http://static.lolesports.com/players/164373420...,108764409368519858,2022-08-04 12:12:30+00:00,2024-07-08 03:07:07+00:00,108764409368519858,REIGNITE Lily,reignite-lily,RIG
2455,108764522500018946,SayaJP,Sayaka,Kojima,active,http://static.lolesports.com/players/164373420...,108764409368519858,2022-08-04 12:12:30+00:00,2024-07-08 03:07:07+00:00,108764409368519858,REIGNITE Lily,reignite-lily,RIG
2456,108764522500018946,SayaJP,Sayaka,Kojima,active,http://static.lolesports.com/players/164373420...,108764409368519858,2022-08-04 12:12:30+00:00,2024-07-08 03:07:07+00:00,108764409368519858,REIGNITE Lily,reignite-lily,RIG


In [91]:
# Get games for a league
games = pd.read_json("game-changers/esports-data/mapping_data.json")

games

,platformGameId,esportsGameId,tournamentId,teamMapping,participantMapping
0,val:a02b939b-d53b-4d6c-9cf9-8df5cbecdec6,110614988238486656,110614878961138672,"{'18': '107185970379663407', '19': '1082407634...","{'1': '110196391536301033', '2': '108550500823..."
1,val:865eb670-8fd8-4245-97b3-44540e321f8e,110780564307690144,110780564306117264,"{'18': '110745562660975168', '19': '1087481697...","{'1': '108748099637064824', '2': '108748093351..."
2,val:4d834f45-c9db-42fa-ae55-e8afd451e434,109879965968343856,109879965964411664,"{'17': '108452390946539707', '18': '1076008816...","{'1': '107600796790972557', '2': '107600798796..."
3,val:178e8b84-61fa-4a14-835d-8f0e0af83466,111006223833455232,111006194588670528,"{'19': '108748177717149112', '20': '1110572219...","{'1': '111067926063836716', '2': '111067931077..."
4,val:03bdf51f-453c-4fab-9406-63beee427e6d,109879965968475072,109879965964411664,"{'17': '108172266878972669', '18': '1098798864...","{'1': '107897747787068260', '2': '107910792460..."
...,...,...,...,...,...
1098,val:d507ec8f-8ab8-4a65-94d6-64f34aa8010e,112472465715573328,112472465714786864,"{'17': '111901707633760155', '18': '1071859703...","{'1': '107025879556684669', '2': '107025879465..."
1099,val:945b723a-0f6d-4243-9d80-24013f39139c,112196265094407568,112093580768495968,"{'16': '112172170311847035', '17': '1121843686...","{'1': '112184652059037339', '2': '112184651995..."
1100,val:77e3feca-b86d-43a2-b01c-3758d16ee7d3,112279699443897088,112279699443110656,"{'18': '108748177717149112', '19': '1087481697...","{'1': '111057371424123924', '2': '108748099637..."
1101,val:0b9b55b0-979b-4c25-8033-8ccdc7a8f451,112279699443897088,112279699443110656,"{'19': '112279706800846545', '20': '1107455626...","{'1': '111057427541732898', '2': '108748092183..."


In [92]:
# Get list of games that the player has played
games_filter = games[games['participantMapping'].apply(lambda mapping: (player_id) in mapping.values())]
# Remove prefix
games_filter['platformGameId'] = games_filter['platformGameId'].apply(lambda gameID: gameID[4:])

player_games = list(games_filter['platformGameId'])


player_games

C:\Users\Angus\AppData\Local\Temp\ipykernel_20264\310192364.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_filter['platformGameId'] = games_filter['platformGameId'].apply(lambda gameID: gameID[4:])


['379881fc-03dc-4b7d-9e8a-ea16c889a983',
 '3d76a564-c73c-48bd-a3e7-8c6d09ccdbc4',
 '2163510b-9b6b-4277-9be2-f4c8ea3ffbd8',
 '4fa5fe65-e566-41b0-9e53-0d97dee261ff',
 '1883e48d-dddb-4270-9ae9-ba430bb240e6',
 'cf5866f5-5d2c-4028-b3d9-03d6e31096f7',
 'aa831837-80ff-410a-b389-ffc8bee3ed60',
 'a789c52b-2365-49da-bc5d-d68f9f226306',
 'e917a8fe-5d41-424d-83ea-83b65bc82098',
 '6aad3a98-a337-4c6d-b654-5691935bee43',
 '08dd9a05-3bb2-4af9-b682-4383dd396e26',
 'bed24775-df28-4350-84e3-9bf79ee55438',
 '6c5ba7a8-c1d9-4458-b875-f2cd0d49dd84',
 'e94d9a7b-c527-403f-a2da-608a83c286f7',
 'd7bc6669-96e0-4800-a8ed-87a7de369f53',
 'a0f51b81-51bc-45b7-a632-e124461c83e6',
 'ebdcb37f-4613-4d09-b051-bb328ee9d3dc',
 '6cb2c087-805f-46d5-9d74-31af7c0882e5',
 '8d68f9d7-ae38-435e-aebb-9ece8daa624e',
 '8fd0c22c-0c69-4fda-a4b8-af47b8cce340',
 '52edd6eb-2916-46d1-8b0a-8d7086f0035a',
 '0a63934c-9907-4b7c-a553-ac945cc9eea4',
 '21cf85ba-8e8e-47c1-b6d5-d35a419c3747',
 'ec1e792f-d4f0-41c5-86f3-728af7e8ae4d',
 'ca21a6a4-31d9-

In [93]:
len(player_games)

34

In [94]:
# Create map of who downed who in a target game
target_game = "0a63934c-9907-4b7c-a553-ac945cc9eea4"

try:
        with open(os.path.join('game-changers/games/2022/', 'val_'+target_game+'.json'), 'r') as file:

                game_data = json.load(file)

except:
        print("No such file")

In [95]:
# Get player mapping to connect player id 
game_player_map = games[games['platformGameId'] == "val:"+target_game]['participantMapping']
game_player_map = game_player_map.to_dict()[game_player_map.index[0]]
game_player_map

{'1': '108764514258014973',
 '2': '108764514005333971',
 '3': '108764513807403457',
 '4': '108764522500018946',
 '5': '108764516777284603',
 '6': '109030080591889737',
 '7': '109030079523193762',
 '8': '109030082839148195',
 '9': '109030083962930086',
 '10': '109030081641666982'}

In [96]:
# Get player deaths

deaths_from = {}

for snapshot in game_data:

    if "playerDied" in snapshot:
        print(snapshot)

        # Check if death is our target player
        died_player = str(snapshot['playerDied']['deceasedId']['value'])
        died_player_id = game_player_map[died_player]

        if died_player_id == player_id:
                

            killer = str(snapshot['playerDied']['killerId']['value'])
            killer_player_id = game_player_map[killer]

            # Check if deaths from player already exist
            if killer_player_id in deaths_from:
                deaths_from[killer_player_id] += 1
            else:
                deaths_from[killer_player_id] = 1


{'platformGameId': 'val:0a63934c-9907-4b7c-a553-ac945cc9eea4', 'playerDied': {'deceasedId': {'value': 10}, 'assistants': [{'assistantId': {'value': 3}}], 'weapon': {'fallback': {'displayName': '', 'inventorySlot': {'slot': 'SECONDARY'}, 'guid': '29A0CFAB-485B-F5D5-779A-B59F85E204A8'}, 'type': 'UNKNOWN'}, 'killerId': {'value': 2}}, 'metadata': {'serverInfo': {'processId': '', 'rfc190Scope': 'aresriot.aws-ape1-prod.tournament-gp-hongkong-1.ares.gameserver'}, 'playback': 0, 'sequenceNumber': 1827, 'stage': 1, 'gameVersion': '05.07.00.775731', 'gameId': {'value': '0a63934c-9907-4b7c-a553-ac945cc9eea4'}, 'eventTime': {'includedPauses': '512.166s', 'omittingPauses': '512.166s'}, 'wallTime': '2022-10-13T09:11:08.835Z'}}
{'platformGameId': 'val:0a63934c-9907-4b7c-a553-ac945cc9eea4', 'playerDied': {'deceasedId': {'value': 5}, 'ability': {'fallback': {'displayName': '', 'guid': '9F0D8BA9-4140-B941-57D3-A7AD57C6B417', 'inventorySlot': {'slot': 'ABILITY_1'}}, 'type': 'UNKNOWN'}, 'assistants': [{'a

In [97]:
print(deaths_from)

{'109030083962930086': 4, '109030079523193762': 3, '109030080591889737': 2, '109030081641666982': 1, '109030082839148195': 1}


In [98]:
game_player_map['1']

'108764514258014973'

In [99]:
# Add elastisearch for searching
games_dir = 'J:/VCT Hackathon Val Tournament Data/game-changers/games/2022/'

search = Elasticsearch()

matching = []

for game in os.listdir(games_dir):

    with open(os.path.join(games_dir, game), 'r') as file:

        game_data = json.load(file)

        search.index(index='json_files', body=game_data)

ValueError: Either 'hosts' or 'cloud_id' must be specified